In [1]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook, tnrange
from itertools import chain
# from skimage.io import imread, imshow, concatenate_images
# from skimage.transform import resize
# from skimage.morphology import label
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D , Flatten,LeakyReLU,BatchNormalization
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from tensorflow.keras.layers import Conv2D, Conv2DTranspose,DepthwiseConv2D,Add,Reshape
from tensorflow.keras.layers import MaxPooling2D, GlobalMaxPool2D, Concatenate
from tensorflow.keras.layers import concatenate, add
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tqdm import tqdm
from keras.models import Model
from keras.layers import Input, Conv2D, GlobalAveragePooling2D, Dropout
#from keras.utils.vis_utils import plot_model
from tensorflow import keras
from keras import backend as K


2024-04-07 20:07:18.172189: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-07 20:07:18.492361: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-07 20:07:18.492409: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-07 20:07:18.548408: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-07 20:07:18.657716: I tensorflow/core/platform/cpu_feature_guar

In [2]:

image_directory = '/home/sagnik/ML_Folder/Large_fish_dataset/Images'
# Define the data directories for images and masks
mask_directory = '/home/sagnik/ML_Folder/Large_fish_dataset/Masks'  # Adjust this path to your mask folder

# Create separate data generators for training and test sets
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Training set (images)
training_image_set = train_datagen.flow_from_directory(
    image_directory,
    target_size=(256, 256),
    batch_size=16,
    class_mode= 'categorical',  # Set to None for masks (segmentation)
    shuffle=False,
    subset='training',
    #color_mode = 'grayscale'
)

# Test set (images)
test_image_set = test_datagen.flow_from_directory(
    image_directory,
    target_size=(256, 256),
    batch_size=16,
    class_mode='categorical',
    shuffle=False,
    subset='validation',
    #color_mode = 'grayscale',
)

# Training set (masks)
training_mask_set = train_datagen.flow_from_directory(
    mask_directory,
    target_size=(256, 256),
    batch_size=16,
    class_mode=None,  # Set to None for masks (segmentation)
    shuffle=False,
    subset='training',
    #color_mode = 'grayscale'
)

# Test set (masks)
test_mask_set = test_datagen.flow_from_directory(
    mask_directory,
    target_size=(256, 256),
    batch_size=16,
    class_mode=None,
    shuffle=False,
    subset='validation'
)

# Now create class labels (0 to 8) for classification
# You can use these labels along with the image data for classification
# ...
# Assuming you have already loaded your data and created the data generators



Found 7200 images belonging to 9 classes.
Found 1800 images belonging to 9 classes.
Found 7200 images belonging to 9 classes.
Found 1800 images belonging to 9 classes.


In [3]:
X_train = training_image_set[0][0]  # Features for training (images)
X_test = test_image_set[0][0]      # Features for testing (images)

y_train1 = training_image_set[0][1]  # Labels for training (masks)
y_test1 = test_image_set[0][1]      # Labels for testing (masks)
# Extract the mask data from the data generators
y_train2 = training_mask_set[0]     # Labels for training (masks)
y_test2 = test_mask_set[0]  

In [4]:
# Assuming you have created your data generator (e.g., training_data_generator)
# Get the class labels (categorical) using class_indices
class_labels = list(training_image_set.class_indices.keys())
print("Class labels:", class_labels)


Class labels: ['Black Sea Sprat', 'Gilt-Head Bream', 'Hourse Mackerel', 'Red Mullet', 'Red Sea Bream', 'Sea Bass', 'Shrimp', 'Striped Red Mullet', 'Trout']


In [5]:
print(y_train2.shape)

(16, 256, 256, 3)


In [6]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3,shuffle=True)

In [7]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    #x = _inverted_residual_block(inputs, num_filters, (3, 3), t=1, alpha=1.0, strides=1, n=1)
    p = MaxPool2D((2, 2))(x)
    print( x.shape  , "\t" , p.shape)
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    print("=================================")
    x = Conv2DTranspose(num_filters, 2, strides=2, padding="same")(inputs)
    print(x.shape, "\t" , skip_features.shape)
    x = Concatenate()([x, skip_features])
    print(x.shape)
    x = conv_block(x, num_filters)
    print(x.shape)
    print("=================================")

    return x

def fully_connected_block(inputs,num_classes):
    if num_classes >= 3:
        Activation = 'softmax'
    elif num_classes ==2:
        Activation = 'sigmoid'
    else:
        print("input correct number of classes")
        
    x = Flatten()(inputs)
    x = BatchNormalization()(x)
    x = Dense(units = 4096 ,activation = 'relu')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Dense(units = 4096 ,activation = 'relu')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Dense(units = num_classes ,activation =Activation ,name = 'class_outputs')(x)
    return x

In [8]:
def Encoder_arm(input_shape):
    inputs = Input(input_shape)
    s0, p0 = encoder_block(inputs, 64)
    s1, p1 = encoder_block(p0, 128)
    s2, p2 = encoder_block(p1, 256)
    s3, p3 = encoder_block(p2, 512)
    s4, p4 = encoder_block(p3, 1024)
    S = [s0,s1,s2,s3,s4]
    P = [p0,p1,p2,p3,p4]
    #print(s4.shape , p4.shape)
    return S,P

def Base(inputs):
    base =conv_block(inputs[4], 2048)
    return base
    
def Decoder_arm(base,skip_connection):
    
    d0 = decoder_block(base, skip_connection[4], 1024)
    d1 = decoder_block(d0, skip_connection[3], 512)
    d2 = decoder_block(d1, skip_connection[2], 256)
    d3 = decoder_block(d2, skip_connection[1], 128)
    d4 = decoder_block(d3, skip_connection[0], 64)
    mask_outputs = Conv2D(1, 1, padding="same", activation="sigmoid" , name = 'mask_outputs')(d4)
    return mask_outputs

def Classification_tail(inputs,num_classes):
    class_outputs = fully_connected_block(inputs[4],num_classes)
    return class_outputs

In [9]:
skip,pool  = Encoder_arm((256,256,3))

2024-04-07 20:07:23.498022: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


(None, 256, 256, 64) 	 (None, 128, 128, 64)
(None, 128, 128, 128) 	 (None, 64, 64, 128)
(None, 64, 64, 256) 	 (None, 32, 32, 256)
(None, 32, 32, 512) 	 (None, 16, 16, 512)
(None, 16, 16, 1024) 	 (None, 8, 8, 1024)


2024-04-07 20:07:23.681033: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-07 20:07:23.681091: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-07 20:07:23.682694: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-07 20:07:23.682753: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-07 20:07:23.682773: I external/local_xla/xla/stream_executor

In [14]:
Base(pool)

<KerasTensor: shape=(None, 8, 8, 2048) dtype=float32 (created by layer 'activation_33')>

In [10]:
Decoder_arm(Base(pool),skip)

(None, 16, 16, 1024) 	 (None, 16, 16, 1024)
(None, 16, 16, 2048)
(None, 16, 16, 1024)
(None, 32, 32, 512) 	 (None, 32, 32, 512)
(None, 32, 32, 1024)
(None, 32, 32, 512)
(None, 64, 64, 256) 	 (None, 64, 64, 256)
(None, 64, 64, 512)
(None, 64, 64, 256)
(None, 128, 128, 128) 	 (None, 128, 128, 128)
(None, 128, 128, 256)
(None, 128, 128, 128)
(None, 256, 256, 64) 	 (None, 256, 256, 64)
(None, 256, 256, 128)
(None, 256, 256, 64)


<KerasTensor: shape=(None, 256, 256, 1) dtype=float32 (created by layer 'mask_outputs')>

In [11]:
def U_NET(input_shape,num_classes):
    #input = Input(shape=input_shape)
    S,P =Encoder_arm(input_shape)
    output1 = Classification_tail(P,num_classes)
    print("---------------------")
    print(output1.shape)
    print("---------------------")
    base = Base(P)
    output2 = Decoder_arm(base,S)
    print("---------------------")
    print(output2.shape)
    print("---------------------")
    
    model = Model(inputs=Input(shape=input_shape) , outputs = [output1,output2], name = "U-Net")
    print(model.summary)
    return model

In [12]:
model =  U_NET((256,256,3),9)

(None, 256, 256, 64) 	 (None, 128, 128, 64)
(None, 128, 128, 128) 	 (None, 64, 64, 128)
(None, 64, 64, 256) 	 (None, 32, 32, 256)
(None, 32, 32, 512) 	 (None, 16, 16, 512)
(None, 16, 16, 1024) 	 (None, 8, 8, 1024)


2024-04-07 20:07:44.742171: W external/local_tsl/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.00GiB (rounded to 1073741824)requested by op AddV2
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-04-07 20:07:44.742230: I external/local_tsl/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2024-04-07 20:07:44.742238: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 75, Chunks in use: 75. 18.8KiB allocated for chunks. 18.8KiB in use in bin. 8.2KiB client-requested in use in bin.
2024-04-07 20:07:44.742242: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 31, Chunks in use: 31. 15.5KiB allocated for chunks. 15.5KiB in use in bin. 15.5KiB client-requested in use in bin.
2024-04-07 20:07:44.7422

ResourceExhaustedError: {{function_node __wrapped__AddV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:AddV2] name: 

In [ ]:
model.summary()

In [ ]:
def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)


def jacard_coef_loss(y_true, y_pred):
    return -jacard_coef(y_true, y_pred)  # -1 ultiplied as we want to minimize this value as loss function

loss1 = keras.losses.SparseCategoricalCrossentropy(from_logits = False)
loss2 = keras.losses.BinaryCrossentropy(from_logits = False)

losses = {
    "class_outputs" : loss1 ,
    "mask_outputs" : loss2
}

#lossWeights = {"category_output": 1.0, "color_output": 1.0}

In [ ]:
# Compile the model with appropriate loss functions
model.compile(optimizer=Adam(),
              loss=losses,
              metrics=['accuracy'])



In [ ]:
print(y_train1.shape,"\t", y_train2.shape)

In [ ]:
# Train the model
history = model.fit(X_train, [y_train1,  y_train2],
                    validation_data=(X_test, [y_test1, y_test2]),
                    epochs=5)


In [ ]:
model.save_weights('/kaggle/working/Gamma.h5')
model.load_weights('/kaggle/working/Gamma.h5')

In [ ]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test, y_test1,y_test2)
print("Test Accuracy:", accuracy)

# Predict masks on test data
y_pred1 , y_pred2 = model.predict(X_test)

# Convert predicted masks to binary values
y_pred_binary = (y_pred1 > 0.5).astype(np.uint8)
y_test1 = (y_test1 > 0.5).astype(np.uint8)



In [ ]:
X_test.shape

In [ ]:
import random
rand_num = random.randint(0,613)
print(rand_num)

In [ ]:
img_demo = X_test[rand_num]
plt.imshow(img_demo)

In [ ]:
seg_demo = y_test2[rand_num]
plt.imshow(seg_demo)

In [ ]:
def dice_coefficient(y_true, y_pred22):
    intersection = np.sum(y_true * y_pred2)
    union = np.sum(y_true) + np.sum(y_pred2)
    dice = (2.0 * intersection) / (union + intersection)
    return dice
dice_scores = []
for i in range(len(X_test)):
    dice = dice_coefficient(y_test2[i], y_pred2_binary[i])
    dice_scores.append(dice)
average_dice = np.mean(dice_scores)
print("Average Dice Score:", average_dice)
